In [1]:
import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import time
import os
from tqdm import tqdm

In [2]:
# 데이터셋이 위치한 경로
# 여러분들이 다운로드 받은 데이터셋의 경로를 찍어주시면 됩니다.
data_dir = './dataset'

In [3]:
# 이미지 변환
# 이리저리 바꿔보면서 좋은 성능이 나오도록 하시면 됩니다.
# 다만 validation은 건들지 말아주세요
class GaussianNoise:
    def __init__(self, mean=0.0, std=1.0):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return f"{self.__class__.__name__}(mean={self.mean}, std={self.std})"
    
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)), #gaussian blur
        transforms.RandomRotation(25),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
        transforms.ToTensor(),
        GaussianNoise(mean=0.0, std=0.1), #gaussian noise
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [4]:
from torch.utils.data import ConcatDataset

# 이미지 데이터셋 불러오기
train_dataset  = datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train'])
pgd_dataset = datasets.ImageFolder(os.path.join(data_dir, 'valid'), data_transforms['train'])

full_dataset = ConcatDataset([train_dataset, pgd_dataset])
                                                                                    
# 클래스명                                                                      
class_names = train_dataset.classes

In [5]:
# 데이터셋을 train과 valid로 나누기
train_size = int(0.7 * len(full_dataset))
valid_size = len(full_dataset) - train_size
train_dataset, valid_dataset = random_split(full_dataset, [train_size, valid_size])

# 데이터 로더
# 만약 CUDA out of Memory가 뜬다면
# 1. batch_size를 줄이거나,
# 2. 모델을 가벼운 걸로 바꾸거나
# 3. data_transforms 에서 Resize 부분을 작게해서 이미지 사이즈를 줄여주세요.
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False, num_workers=2)

In [6]:
# GPU 사용 가능 확인
# device(type='cuda', index=0) 가 뜬다면 GPU에서 훈련이 가능해지고 더 빨라집니다.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [7]:
# pretrained 모델 불러오기
# 여러분이 원하는 모델을 여기서 설정해주세요
from torchvision.models import ResNet101_Weights

model = models.resnet101(weights=ResNet101_Weights.DEFAULT)

# 새로운 분류기를 마지막 레이어에 추가
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

# 모델 GPU에 올리기
model = model.to(device)

In [8]:
# 손실 함수와 optimizer 설정
# optimizer도 다양하게 바꿔가면서 해보시면 좋습니다. 특히 lr (학습률) 관련해서 잘 설정해주세요
from torch.optim import Adam

criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=3.98006e-05)

# scheduler 설정
# torch.optim.lr_scheduler에 보면 다양한 scheduler가 존재합니다. 한번 확인해보고 다양하게 시도해보세요
step_lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
# epoch 횟수 설정
num_epochs=50

In [9]:
# 모델 학습과 평가
# 이 부분은 건드셔도 되고 안건드셔도 됩니다.
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    print('-' * 10)
    for phase in ['train', 'valid']:
        if phase == 'train':
            dataloader = train_loader
            model.train()
        else:
            dataloader = valid_loader
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in tqdm(dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        if phase == 'train':
            step_lr_scheduler.step()

        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_acc = running_corrects.double() / len(dataloader.dataset)

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(
            phase, epoch_loss, epoch_acc))

print('Training complete')

Epoch 1/1
----------


100% 117/117 [01:36<00:00,  1.21it/s]


train Loss: 5.2803 Acc: 0.0103


100% 50/50 [00:26<00:00,  1.88it/s]

valid Loss: 5.2189 Acc: 0.0169
Training complete


In [10]:
# 전체 모델 저장하는 부분입니다.
# PATH에는 저장할 경로를 지정해주세요
PATH = './'
torch.save(model, PATH + 'team1_model.pt')

In [11]:
#model = torch.load('model.pt')
#model.eval()  # 평가 모드로 설정
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model.to(device)

In [12]:
from torch.utils.data import DataLoader, Dataset  

class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
            
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = [os.path.join(root_dir, fname) for fname in os.listdir(root_dir) if fname.endswith(('.jpg', '.jpeg', '.png'))]
        
    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert('RGB')
    
        if self.transform:
            image = self.transform(image)
        
        return image, img_path  # 이미지와 파일 경로 반환


In [13]:
from PIL import Image
import pandas as pd

test_dataset = CustomImageDataset(root_dir='./Test/Unknown', transform=data_transforms['val'])
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 예측 수행
predictions = []
file_paths = []
for images, paths in test_loader:
    images = images.to(device)
    with torch.no_grad():
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy())
        file_paths.extend(paths)  # 경로 저장

# 결과 저장
file_names = [os.path.basename(path) for path in file_paths]
predicted_labels = [class_names[pred] for pred in predictions] 

# pandas 데이터프레임으로 저장
results_df = pd.DataFrame({
    'File Name': file_names,
    'Predicted Label': predicted_labels
})
results_df.to_csv('label_table.csv', index=False)